In [8]:
import folium
import json
plt.rc('font', family='AppleGothic')

In [18]:
#importing data
from glob import glob
file_list=glob("/Users/yeonghwanchoi/Documents/git/DS_class/data/seoul_crime/*")
crime_total_table=[]

for i, data in enumerate(file_list):
    df=pd.read_csv(data, encoding='euc-kr')
    df['year']=file_list[i].split(".")[0][-7:-3]
    df.dropna(inplace=True)
    crime_total_table.append(df)

crime_total=pd.concat(crime_total_table,ignore_index=True)
crime_raw_data=crime_total[crime_total.columns[:5]].sort_values("year")

In [19]:
#translating kor to eng
crime_raw_data=crime_raw_data.rename(columns={"구분":'class',"죄종":'crime_name',"발생검거":'result',"건수":'count'})

class_name=list(crime_raw_data["class"].unique())
class_list=["Jungbu","Jongno","Namdaemoon", "Seodaemun", "Hyehwa", "Yongsan", "Seongbuk", "Dongdaemun", "Mapo", "Yeongdeungpo", "Seongdong", "Dongjak", "Gwangjin", "Seobu", "Gangbuk", "Geumcheon", "Jungnang", "Gangnam","Gwanak", "Gangseo", "Gangdong","Jongam", "Guro","Seocho", "Yangcheon", "Songpa","Nowon", "Bangbae", "Eunpyeong","Dobong","Suseo"]
class_dict=dict(zip(class_name,class_list))

crime_name=list(crime_raw_data["crime_name"].unique())
crime_name_dict=dict(zip(crime_name,["merder","burglar","rape","theft","violence"]))

crime_result_dict={"발생":"incident","검거":"arrest"}

#implementing translation
input_list=[class_dict,crime_name_dict,crime_result_dict]
columns=crime_raw_data.columns

for i, dicts in enumerate(input_list):
    crime_raw_data[columns[i]]=crime_raw_data[columns[i]].replace(dicts)

In [20]:
#preprocessing data

# transforming police stations to area 
station_name=["Hyehwa","namdaemoon","Jungbu","Jongam","Seobu","Bangbae","Suseo"]
area_name = ["jongno","Jungbu","Jungbu","Seongbuk","Eunpyeong","Seocho","Gangnam"]
tmp_dict=dict(zip(station_name,area_name)) 

crime_raw_data["class"]=crime_raw_data["class"].replace(tmp_dict)

#pivot_table
#trenforming dataframe to pivot_table sorted by year and class 
crime_pivot = crime_raw_data.pivot_table(
    crime_raw_data, index=["year","class"], columns=["crime_name","result"], aggfunc=[np.sum])
crime_pivot.columns = crime_pivot.columns.droplevel([0,1])

#simplify class 
tmp = crime_pivot.columns.get_level_values(0)+"_"+crime_pivot.columns.get_level_values(1)
crime_pivot.columns = tmp

#adding arrest rates columns on dataframe
arrest=["rape_arrest","burglar_arrest","merder_arrest","theft_arrest","violence_arrest"]
occurs=["rape_incident","burglar_incident","merder_incident","theft_incident","violence_incident"]
new_columns=["rape_arrest_rate","burglar_arrest_rate","merder_arrest_rate","theft_arrest_rate","violence_arrest_rate"]
crime_pivot[new_columns]=crime_pivot[occurs]/crime_pivot[arrest].values*100

#simplify pivot table
crime_pivot= crime_pivot.drop(arrest,axis=1)
crime_pivot[crime_pivot[new_columns]>100]=100

#rename columns 
crime_pivot.rename(columns={
    "rape_incident" : "rape",
    "theft_incident" : "theft",
    "merder_incident" : "merder",
    "burglar_incident" : "burglar",
    "violence_incident" : "violence"},
    inplace = True)

### 1. how often have the viorent crimes occurred in each area?

In [21]:
#normalization by each years
col=["merder","burglar","rape","theft","violence"]
col2= ["merder_arrest_rate","burglar_arrest_rate","rape_arrest_rate","theft_arrest_rate","violence_arrest_rate"]
years=list(set([i for i , _ in crime_pivot.index]))

for year in years:
    crime_pivot.loc[(year)][col]=crime_pivot.loc[(year)][col]/ crime_pivot.loc[(year)][col].max()
crime_pivot

#adding average crime and arrest rate on area 
crime_pivot["mean_crime"] = np.mean(crime_pivot[col],axis=1)
crime_pivot["mean_arrest"] = np.mean(crime_pivot[col2],axis=1)
crime_pivot=crime_pivot.round(2)
crime_pivot

burglar  merder  rape  theft  violence  rape_arrest_rate  \
year class                                                                    
2001 Dobong           0.26    0.45  0.37   0.27      0.32             100.0   
     Dongdaemun       0.41    0.27  0.48   0.41      0.31             100.0   
     Dongjak          0.29    0.09  0.34   0.33      0.40             100.0   
     Eunpyeong        0.89    0.73  0.84   0.62      0.83             100.0   
     Gangbuk          0.50    0.14  0.33   0.46      0.22             100.0   
...                    ...     ...   ...    ...       ...               ...   
2017 Songpa           0.53    0.40  0.41   1.00      0.91             100.0   
     Yangcheon        0.44    0.47  0.52   0.62      0.66             100.0   
     Yeongdeungpo     0.57    0.40  0.39   1.00      0.52             100.0   
     Yongsan          0.53    0.20  0.49   0.62      1.00             100.0   
     jongno           0.39    0.73  0.41   0.23      0.33             100.0   

                   burglar_arrest_rate  merder_arrest_rate  theft_arrest_rate  \
year class                                                                      
2001 Dobong                      100.0               83.33              82.00   
     Dongdaemun                  100.0              100.00              89.86   
     Dongjak                     100.0               66.67             100.00   
     Eunpyeong                   100.0               88.89             100.00   
     Gangbuk                     100.0              100.00             100.00   
...                                ...                 ...                ...   
2017 Songpa                      100.0              100.00              81.25   
     Yangcheon                   100.0               87.50             100.00   
     Yeongdeungpo                100.0              100.00             100.00   
     Yongsan                     100.0              100.00             100.00   
     jongno                      100.0              100.00              60.00   

                   violence_arrest_rate  mean_crime  mean_arrest  
year class                                                        
2001 Dobong                       100.0        0.33        93.07  
     Dongdaemun                   100.0        0.37        97.97  
     Dongjak                      100.0        0.29        93.33  
     Eunpyeong                    100.0        0.78        97.78  
     Gangbuk                      100.0        0.33       100.00  
...                                 ...         ...          ...  
2017 Songpa                       100.0        0.65        96.25  
     Yangcheon                    100.0        0.54        97.50  
     Yeongdeungpo                 100.0        0.58       100.00  
     Yongsan                      100.0        0.57       100.00  
     jongno                       100.0        0.42        92.00  

[459 rows x 12 columns]

### visualization

In [22]:
path='/Users/yeonghwanchoi/Documents/git/DS_class/data/03_skorea_municipalities_geo_simple.json'
geo_str=json.load(open(path, encoding="utf-8"))


In [28]:
my_map = folium.Map(location=[37.5502,126.982],zoom_start=11,titles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=crime_pivot, columns=["class","merder"]
)

KeyError: "None of ['class'] are in the columns"

In [ ]:
import json
geo_path = "/Users/yeonghwanchoi/Documents/git/DS_class/data/03_skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path,encoding="utf-8"))

In [17]:
geo_str

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '강동구',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.16683184366129, 37.57672487388627],
      [127.18408792330152, 37.55814280369575],
      [127.16530984307447, 37.54221851258693],
      [127.14672806823502, 37.51415680680291],
      [127.12123165719615, 37.52528270089],
      [127.1116764203608, 37.540669955324965],
      [127.11519584981606, 37.557533180704915]]]}},
  {'type': 'Feature',
   'id': '송파구',
   'properties': {'code': '11240',
    'name': '송파구',
    'name_eng': 'Songpa-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.0690698130372, 37.522279423505026],
      [127.10087519791962, 37.524841220167055],
      [127.1116764203608, 37.540669955324965],
      [127.12123165719615, 37.52528270089

In [1]:
a=[1,2,3,4,5]
a[5]

IndexError: list index out of range